<a href="https://colab.research.google.com/github/rulogar1234/nuclio-1/blob/develop/SUP_ML_3v1_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUP ML 3 - PREDICT

# Librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
from sklearn.pipeline import Pipeline,FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn import set_config
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder


pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#para que las salidas sean dataframes
set_config(transform_output = "pandas")

In [ ]:
PATH_DATA='/content/drive/MyDrive/Colab Notebooks/TEMPLATE SUP ML (main)/EJERCICIO-ML-SUP/data/'
modelo_pickle_=PATH_DATA+'modelo_pickle.pkl'
model = pickle.load(open(modelo_pickle_, 'rb'))

# Carga PREDICT dataset

In [ ]:
ruta_predict_='/content/drive/MyDrive/Colab Notebooks/TEMPLATE SUP ML (main)/EJERCICIO-ML-SUP/data/telecom_churn_PREDICT.csv'
df=pd.read_csv(ruta_predict_)
df.shape

(10000, 99)

In [ ]:
df.describe()

,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,change_rev,drop_vce,drop_dat,blck_vce,blck_dat,unan_vce,unan_dat,plcd_vce,plcd_dat,recv_vce,recv_sms,comp_vce,comp_dat,custcare,ccrndmou,cc_mou,inonemin,threeway,mou_cvce,mou_cdat,mou_rvce,owylis_vce,mouowylisv,iwylis_vce,mouiwylisv,peak_vce,peak_dat,mou_peav,mou_pead,opk_vce,opk_dat,mou_opkv,mou_opkd,drop_blk,attempt,complete,callfwdv,callwait,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,Customer_ID
count,9777.000000,9777.000000,9777.000000,9777.000000,9777.000000,9777.000000,9777.000000,9777.000000,9777.000000,9704.000000,9704.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,8568.000000,8568.000000,8568.000000,9857.000000,10000.00000,10000.000000,9719.000000,9719.000000,6312.000000,7014.000000,6760.00000,4202.000000,9719.000000,10000.000000,1.000000e+04
mean,62.461781,598.876027,48.922510,0.990076,43.584032,16.037868,15.583912,0.450038,1.086100,-15.286609,-0.527241,7.624100,0.064667,4.234133,0.080133,36.257000,0.047800,176.438700,1.456567,67.431833,0.045867,130.577533,1.263967,3.069600,7.565133,5.831717,37.484167,0.421500,285.359657,3.231364,134.903334,29.115233,34.608442,9.535567,22.844159,101.808533,0.567000,199.024314,1.189307,83.099400,0.696967,221.082699,2.042054,12.003033,177.895267,131.841500,0.023267,2.233933,14.965200,1.485800,1.29090,2035.394400,5608.849200,727.790452,650.197828,5471.883700,1981.786100,63.543227,588.794152,200.425271,601.553500,199.330200,62.707300,592.727591,199.511204,62.994631,121.499577,1.64430,1.452600,0.112563,0.043523,6.140843,2.451811,5.52071,1.552832,0.046301,297.549200,1.095000e+06
std,47.998705,594.887840,25.672879,2.211176,100.208979,33.675211,32.902230,4.020511,5.752155,314.247193,48.457858,11.034259,0.798077,11.270139,4.219658,45.853533,0.456317,178.321647,13.264834,96.927743,1.622859,131.205445,11.399759,6.494457,17.104817,14.033230,62.631408,1.688303,316.161961,36.219368,182.900381,38.364826,56.922499,18.055273,49.568753,108.754511,5.387241,222.342059,14.926829,104.144780,6.528905,296.623589,23.644297,18.053678,180.087903,132.820402,1.021321,6.113823,12.738313,0.871937,0.59926,2622.322296,6199.930036,658.029605,650.711233,6052.855441,2550.683585,41.753937,531.845920,188.222629,601.740077,203.314102,48.485112,554.990296,192.780061,43.268317,59.444070,1.25783,0.870993,0.316074,0.204042,4.674948,1.452279,2.21680,0.592898,0.210147,269.575481,2.886896e+03
min,-6.167500,0.000000,-6.167500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3041.250000,-458.972500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,1.000000,0.00000,0.000000,0.000000,3.650000,2.400000,0.000000,0.000000,0.480000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,9.989998,1.00000,1.000000,0.000000,0.000000,0.000000,1.000000,1.00000,1.000000,0.000000,-4.000000,1.090001e+06
25%,35.310000,176.333333,33.742500,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 99 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               9777 non-null   float64
 1   mou               9777 non-null   float64
 2   totmrc            9777 non-null   float64
 3   da                9777 non-null   float64
 4   ovrmou            9777 non-null   float64
 5   ovrrev            9777 non-null   float64
 6   vceovr            9777 non-null   float64
 7   datovr            9777 non-null   float64
 8   roam              9777 non-null   float64
 9   change_mou        9704 non-null   float64
 10  change_rev        9704 non-null   float64
 11  drop_vce          10000 non-null  float64
 12  drop_dat          10000 non-null  float64
 13  blck_vce          10000 non-null  float64
 14  blck_dat          10000 non-null  float64
 15  unan_vce          10000 non-null  float64
 16  unan_dat          10000 non-null  float64

In [ ]:
df.head()

,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,change_rev,drop_vce,drop_dat,blck_vce,blck_dat,unan_vce,unan_dat,plcd_vce,plcd_dat,recv_vce,recv_sms,comp_vce,comp_dat,custcare,ccrndmou,cc_mou,inonemin,threeway,mou_cvce,mou_cdat,mou_rvce,owylis_vce,mouowylisv,iwylis_vce,mouiwylisv,peak_vce,peak_dat,mou_peav,mou_pead,opk_vce,opk_dat,mou_opkv,mou_opkd,drop_blk,attempt,complete,callfwdv,callwait,months,uniqsubs,actvsubs,new_cell,crclscod,asl_flag,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,prizm_social_one,area,dualband,refurb_new,hnd_price,phones,models,hnd_webcap,truck,rv,ownrent,lor,dwlltype,marital,adults,infobase,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,30.8350,136.75,29.9900,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,1.1550,2.000000,0.000000,10.333333,0.0,13.333333,0.0,66.000000,0.000000,3.333333,0.0,40.666667,0.000000,1.333333,1.333333,0.896667,0.666667,0.333333,71.746667,0.000000,5.406667,1.333333,0.713333,0.333333,0.473333,30.000000,0.0,54.500000,0.0,13.666667,0.000000,22.653333,0.000000,12.333333,66.000000,40.666667,0.0,0.000000,7,1,1,U,AA,N,393,958.0,248.46,182.94,958.0,393,30.49,159.67,65.50,121,52,30,160.0,66.0,31.0,C,ATLANTIC SOUTH AREA,Y,N,149.98999,1.0,1.0,WCMB,0.0,0.0,R,NaN,M,M,1.0,M,3.0,NaN,C,NaN,1.0,U,U,U,U,U,U,Y,216.0,1090001
1,35.8475,352.75,24.2700,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,-35.8475,9.000000,0.000000,0.000000,0.0,10.000000,0.0,82.666667,1.000000,31.000000,0.0,65.333333,1.000000,3.666667,9.000000,5.890000,18.333333,1.333333,98.543333,1.023333,83.666667,15.666667,12.683333,1.333333,14.286667,38.000000,0.0,106.926667,0.0,43.666667,1.000000,75.283333,1.023333,9.000000,83.666667,66.333333,0.0,1.333333,7,1,1,U,EA,Y,1641,4011.0,391.87,305.59,3998.0,1634,50.93,666.33,272.33,470,181,48,666.0,272.0,51.0,C,NORTHWEST/ROCKY MOUNTAIN AREA,T,N,99.98999,2.0,1.0,WCMB,0.0,0.0,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,0.0,N,U,U,U,U,U,N,101.0,1090002
2,30.3275,241.50,39.9900,0.0000,0.00,0.000,0.0,0.000,0.0000,-86.50,0.8625,2.000000,0.000000,4.666667,0.0,20.666667,0.0,85.333333,0.000000,26.666667,0.0,59.333333,0.000000,4.333333,6.333333,4.466667,17.000000,1.666667,143.530000,0.000000,27.373333,9.666667,3.760000,19.666667,21.736667,48.666667,0.0,76.356667,0.0,36.000000,0.000000,94.543333,0.000000,6.666667,85.333333,59.333333,0.0,0.333333,9,2,2,Y,A,N,780,1889.0,312.91,219.45,1772.0,734,31.35,253.14,104.86,270,107,30,273.0,109.0,31.0,T,CENTRAL/SOUTH TEXAS AREA,N,N,129.98999,1.0,1.0,WCMB,1.0,0.0,O,13.0,S,M,3.0,M,6.0,1.0,C,A,0.0,N,U,U,U,U,U,Y,262.0,1090003
3,154.6925,2297.00,149.9900,4.7025,0.00,0.000,0.0,0.000,0.0000,119.00,0.2475,20.333333,0.000000,5.000000,0.0,329.333333,0.0,1067.000000,0.000000,628.000000,0.0,723.000000,0.000000,0.333333,0.666667,0.376667,408.000000,0.000000,769.136667,0.000000,679.020000,271.000000,167.090000,68.666667,150.656667,723.000000,0.0,961.736667,0.0,346.000000,0.000000,486.420000,0.000000,25.333333,1067.000000,723.000000,0.0,8.333333,11,1,1,U,AA,N,9018,18478.0,2217.05,2131.17,18478.0,9018,213.12,1847.80,901.80,2257,1045,155,2452.0,1171.0,278.0,S,ATLANTIC SOUTH AREA,N,R,79.98999,2.0,1.0,WCMB,0.0,0.0,O,3.0,S,U,3.0,M,6.0,1.0,NaN,A,0.0,S,U,U,U,U,U,Y,127.0,1090004
4,156.0050,542.25,48.9475,0.7425,70.75,28.295,28.1,0.195,0.0000,195.75,40.0150,5.666667,0.333333,0.666667,0.0,46.333333,0.0,167.666667,0.666667,11.333333,0.0,115.000000,0.333333,21.666667,48.000000,31.346667,10.000000,0.333333,343.066667,0.010000,11.243333,13.000000,14.563333,1.333333,2.100000,91.000000,0.0,205.216667,0.0,31.000000,0.333333,149.096667,0.010000,6.666667,168.333333,115.333333,0.0,0.666667,9,1,1,U,CA,Y,791,2601.0,879.51,608.66,2601.0,791,86.95,371.57,113.00,477,130,143,349.0,103.0,91.0,U,LOS ANGELES AREA,Y,N,149.98999,2.0,1.0,WCMB,0.0,0.0,O,1.0,NaN,S,1.0,M,6.0,1.0,C,NaN,0.0,N,U,U,U,U,U,Y,37.0,1090005


# ML Preprocessing

Se deben realizar las mismas transformaciones que se aplicaron en el preprocessing del train:
  * Eliminar mismas variables
  * Imputar mismos valores a nulos
  * Aplicar mismo encoding de categoricos
  * ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBEN repetir:
  * Analisis visual de los datos
  * Analisis de correlaciones
  * Analisis de varianza
  * Calculo de metricas que varien con la distribución

In [ ]:
df.set_index('Customer_ID',inplace=True)

In [ ]:
target = 'churn'
features = df.columns[df.columns!=target]

# Divide dataset en matriz de variables (X) y output (y)
X = df[features]

In [ ]:
# Check duplicados
df.index.nunique() != len(df.index)

False

In [ ]:
def check_data_types(df):
    for col_name, col_type in df.dtypes.items():
        col_data = df[col_name]
        if col_type == np.float64 and np.issubdtype(col_data.dtype, np.integer):
            print(f"La columna {col_name} es de tipo float64 pero solo contiene datos de tipo int64")
            return 1
        elif col_type == np.int64 and np.issubdtype(col_data.dtype, np.floating):
            print(f"La columna {col_name} es de tipo int64 pero solo contiene datos de tipo float64")
            return 1
        elif col_type == np.object_ and np.issubdtype(col_data.dtype, np.datetime64):
            print(f"La columna {col_name} es de tipo object pero solo contiene datos de tipo datetime64")
            return 1
        elif col_type == np.datetime64 and np.issubdtype(col_data.dtype, np.object_):
            print(f"La columna {col_name} es de tipo datetime64 pero solo contiene datos de tipo object")
            return 1
        elif col_type != col_data.dtype:
            print(f"La columna {col_name} es de tipo {col_type} pero contiene datos de tipo {col_data.dtype}")
            return 1
    return 0

In [ ]:
check_data_types(df)

0

In [ ]:
#Borramos los mismos campos
df.drop('infobase', axis=1, inplace=True)
df.drop('HHstatin', axis=1, inplace=True)
df.drop('truck', axis=1, inplace=True)
df.drop('models', axis=1, inplace=True)
df.drop('rv', axis=1, inplace=True)
df.drop('lor', axis=1, inplace=True)
df.drop('dwlltype', axis=1, inplace=True)
df.drop('numbcars', axis=1, inplace=True)
df.drop('forgntvl', axis=1, inplace=True)
df.drop('ownrent', axis=1, inplace=True)
df.drop('adults', axis=1, inplace=True)
df.drop('dwllsize', axis=1, inplace=True)
df.drop('ethnic', axis=1, inplace=True)
df.drop('avg6mou', axis=1, inplace=True)
df.drop('avg6qty', axis=1, inplace=True)
df.drop('avg6rev', axis=1, inplace=True)
df.drop('dualband', axis=1, inplace=True)

In [ ]:
df2=df.copy()

In [ ]:
def columnas_con_nulos(dataframe):
    columnas_nulas = dataframe.columns[dataframe.isnull().any()].tolist()
    return columnas_nulas
def crear_dataframe_nulos(dataframe):
    columnas_nulas = columnas_con_nulos(dataframe)
    df_nulos = dataframe[columnas_nulas].copy()
    return df_nulos

# Crear el DataFrame con las columnas que tienen valores nulos y sus valores correspondientes para poder trabajar con undataset más pequeño
df_nulos = crear_dataframe_nulos(df2)

In [ ]:
df_nulos.shape

(10000, 22)

In [ ]:
df_nulos.isnull().sum()

rev                  223
mou                  223
totmrc               223
da                   223
ovrmou               223
ovrrev               223
vceovr               223
datovr               223
roam                 223
change_mou           296
change_rev           296
prizm_social_one     851
hnd_price            143
hnd_webcap           670
marital              281
income              3240
kid0_2               281
kid3_5               281
kid6_10              281
kid11_15             281
kid16_17             281
creditcd             281
dtype: int64

In [ ]:
valores_clasif_num=df_nulos.select_dtypes(include=['float64','int64']).columns.tolist()
print(valores_clasif_num)

['rev', 'mou', 'totmrc', 'da', 'ovrmou', 'ovrrev', 'vceovr', 'datovr', 'roam', 'change_mou', 'change_rev', 'hnd_price', 'income']


In [ ]:
#Vamos a quitar hnd_price y a plicar la misma media que en train
valores_clasif_num.remove('hnd_price')
media_hnd_price = 99.70957487428218

In [ ]:
#creamos una lista con los nombres de las columnas clasificatorias con nulos
indice_clasif=df_nulos.select_dtypes(include='object').columns.tolist()
indice_clasif

['prizm_social_one',
 'hnd_webcap',
 'marital',
 'kid0_2',
 'kid3_5',
 'kid6_10',
 'kid11_15',
 'kid16_17',
 'creditcd']

In [ ]:
#Hacemos una función para separar las variables que deben tener U y las que deben tener Unknown y las que deberán tratarse con un ordinal (kids)
valores_clasif_1=[]
valores_clasif_mas=[]
valores_clasif_kid=[]
for i in indice_clasif:
  l=df_nulos[i].str.len().max()
  kid=i.startswith("kid")
  if l == 1 and kid==False:
    valores_clasif_1.append(i)
  elif l == 1 and kid==True:
    valores_clasif_kid.append(i)
  else:
    valores_clasif_mas.append(i)

print(valores_clasif_1)

['prizm_social_one', 'marital', 'creditcd']


In [ ]:
#añadimos el area todo y que no tenga nulos para igualar el nombre
valores_clasif_mas.append('area')
print(valores_clasif_mas)

['hnd_webcap', 'area']


In [ ]:
print(valores_clasif_kid)

['kid0_2', 'kid3_5', 'kid6_10', 'kid11_15', 'kid16_17']


In [ ]:
#Funciones
def income_outlayers_assign(X):
    X.loc[X['income'].isnull(), 'income'] = outlayer_num
    X.loc[X['income'].notnull(), 'income'] = X['income']
    return X

def eqpdays_outlayers_assign(X):
    X.loc[X['eqpdays'] < 0, 'eqpdays'] = outlayer_num
    X.loc[X['eqpdays'] >= 0, 'eqpdays'] = X['eqpdays']
    return X

#creamos una función para hacer tratamiento ordinal del campo tipo kids
def kids0_2_outlayers_assign(X):
  X.loc[X['kid0_2'] == 'Y','kid0_2'] = 1
  X.loc[X['kid0_2'] == 'U','kid0_2'] = 0
  X.loc[X['kid0_2'].isnull(),'kid0_2'] = X['kid0_2']
  return X

def kids3_5_outlayers_assign(X):
  X.loc[X['kid3_5'] == 'Y','kid3_5'] = 1
  X.loc[X['kid3_5'] == 'U','kid3_5'] = 0
  X.loc[X['kid3_5'].isnull(),'kid3_5'] = X['kid3_5']
  return X

def kids6_10_outlayers_assign(X):
  X.loc[X['kid6_10'] == 'Y','kid6_10'] = 1
  X.loc[X['kid6_10'] == 'U','kid6_10'] = 0
  X.loc[X['kid6_10'].isnull(),'kid6_10'] = X['kid6_10']
  return X

def kids11_15_outlayers_assign(X):
  X.loc[X['kid11_15'] == 'Y','kid11_15'] = 1
  X.loc[X['kid11_15'] == 'U','kid11_15'] = 0
  X.loc[X['kid11_15'].isnull(),'kid11_15'] = X['kid11_15']
  return X

def kids16_17_outlayers_assign(X):
  X.loc[X['kid16_17'] == 'Y','kid16_17'] = 1
  X.loc[X['kid16_17'] == 'U','kid16_17'] = 0
  X.loc[X['kid16_17'].isnull(),'kid16_17'] = X['kid16_17']
  return X




In [ ]:
#asignamos el mismo outlayer numérico que en train
outlayer_num=-3875000

In [ ]:
impute = ColumnTransformer(
    transformers=[
        ("col_1",SimpleImputer(strategy="constant",fill_value='-U'),valores_clasif_1),
        ("col_mas",SimpleImputer(strategy="constant",fill_value='-Unknown'),valores_clasif_mas),
        ("col_num",SimpleImputer(strategy="constant",fill_value=outlayer_num),valores_clasif_num),
        ("hnd_price",SimpleImputer(strategy="constant",fill_value=media_hnd_price),["hnd_price"]),
        ("income", FunctionTransformer(func=income_outlayers_assign),["income"]),
        ("eqpdays",FunctionTransformer(func=eqpdays_outlayers_assign),['eqpdays']),
        ("kid_1",FunctionTransformer(func=kids0_2_outlayers_assign),['kid0_2']),
        ("kid_2",FunctionTransformer(func=kids3_5_outlayers_assign),['kid3_5']),
        ("kid_3",FunctionTransformer(func=kids6_10_outlayers_assign),['kid6_10']),
        ("kid_4",FunctionTransformer(func=kids11_15_outlayers_assign),['kid11_15']),
        ("kid_5",FunctionTransformer(func=kids16_17_outlayers_assign),['kid16_17'])

    ],
    remainder='passthrough'
)

In [ ]:
df_t=impute.fit_transform(df2)

In [ ]:
def prizm_social_one_unkw(X):
    X.loc[X['col_1__prizm_social_one'] == '-U', 'col_1__prizm_social_one'] = 'U'
    X.loc[X['col_1__prizm_social_one'] != '-U', 'col_1__prizm_social_one'] = X['col_1__prizm_social_one']
    return X

def hnd_webcap_unkw(X):
    X.loc[X['col_mas__hnd_webcap'] == '-Unknown', 'col_mas__hnd_webcap'] = 'UNKW'
    X.loc[X['col_mas__hnd_webcap'] != '-Unknown', 'col_mas__hnd_webcap'] = X['col_mas__hnd_webcap']
    return X

def marital_unkw(X):
    X.loc[X['col_1__marital'] == '-U', 'col_1__marital'] = 'U'
    X.loc[X['col_1__marital'] != '-U', 'col_1__marital'] = X['col_1__marital']
    return X

#se ha procedido a tratar cada columan separadamente
def kid_assign(X):
  resultado =  X['kid_1__kid0_2']+X['kid_2__kid3_5']+X['kid_3__kid6_10']+X['kid_4__kid11_15']+X['kid_5__kid16_17']
  return pd.DataFrame(resultado, columns=['kids'])

In [ ]:
impute2 = ColumnTransformer(
    transformers=[
         ("kids",FunctionTransformer(func=kid_assign),['kid_1__kid0_2','kid_2__kid3_5','kid_3__kid6_10','kid_4__kid11_15','kid_5__kid16_17']),
         ("crclscod",OrdinalEncoder(),['remainder__crclscod']),
         ("asl_flag",OrdinalEncoder(),['remainder__asl_flag']),
         ("refurb_new",OrdinalEncoder(),['remainder__refurb_new']),
         ("prizm_social_one",FunctionTransformer(func=prizm_social_one_unkw),['col_1__prizm_social_one']),
         ("hnd_webcap",FunctionTransformer(func=hnd_webcap_unkw),['col_mas__hnd_webcap']),
         ("marital",FunctionTransformer(func=marital_unkw),['col_1__marital']),
         ("area",OneHotEncoder(sparse_output=False),["col_mas__area"]),
         ("new_cell",OneHotEncoder(sparse_output=False),["remainder__new_cell"]),
         ("creditcd",OneHotEncoder(sparse_output=False),["col_1__creditcd"])
    ],
    remainder='passthrough'
)


In [ ]:
df_t2=impute2.fit_transform(df_t)

In [ ]:
impute3 = ColumnTransformer(
    transformers = [
        ("hnd_web_cap",OneHotEncoder(sparse_output=False),["hnd_webcap__col_mas__hnd_webcap"]),
        ("prizm_social_one",OneHotEncoder(sparse_output=False),["prizm_social_one__col_1__prizm_social_one"]),
        ("marital",OneHotEncoder(sparse_output=False),["marital__col_1__marital"]),
        ("kids",SimpleImputer(strategy="constant",fill_value=outlayer_num),["kids__kids"])
    ],
    remainder="passthrough"

)

In [ ]:
df_t3=impute3.fit_transform(df_t2)

In [ ]:
#creamos la funcón para transformar las dos columnas que son numeros(col_kid__kids y dwllsize__col_1__dwllsize) a integers y la web_cap usaremos ahora sí un OneHotEncoder
def cambiar_columna_int(column):
    return column.astype('int64')

In [ ]:
#pasamos kids a integer
impute4 = ColumnTransformer(
    transformers = [
        ("kids",FunctionTransformer(func=cambiar_columna_int),['kids__kids__kids'])
    ],
    remainder="passthrough"

)

In [ ]:
df_t4=impute4.fit_transform(df_t3)

In [ ]:
pipe = Pipeline(
    steps = [
        ("imputer1",impute),
        ("imputer2",impute2),
        ("imputer3",impute3),
        ("imputer4",impute4)
    ]
)

In [ ]:
df_t4=pipe.fit_transform(df2)

In [ ]:
df_t4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1090001 to 1100000
Columns: 110 entries, kids__kids__kids__kids to remainder__remainder__remainder__remainder__phones
dtypes: float64(101), int64(9)
memory usage: 8.7 MB


In [ ]:
def ConvertFloatsToInt(X):
  lista_columnas=[]
  for i in X:
      if X[i].dtype.kind =="f" and np.modf(X[i])[0].sum()==0 and X[i].isnull().sum()==0:
        lista_columnas.append([i])
        X[i]=X[i].astype('int64')
  print(lista_columnas)
  return X

In [ ]:
df_t5 = ConvertFloatsToInt(df_t4)

[['remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_UNKW'], ['remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_WC'], ['remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_WCMB'], ['remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_C'], ['remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_R'], ['remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_S'], ['remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_T'], ['remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_U'], ['remainder__marital__marital__col_1__marital_A'], ['remainder__marital__marital__col_1__marital_B'], ['remainder__marital__marital__col_1__marital_M'], ['remainder__marital__marital__col_1__marital_S'], ['remainder__marital__marital__col_1__marital_U'], ['remainder__remainder__crclscod__remainder__crclscod'], ['remainder__remainder__asl_flag__remainder__asl_flag'], ['remainder__remainder__refurb_new__remainde

In [ ]:
df_t5.columns.to_list()

['kids__kids__kids__kids',
 'remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_UNKW',
 'remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_WC',
 'remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_WCMB',
 'remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_C',
 'remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_R',
 'remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_S',
 'remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_T',
 'remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_U',
 'remainder__marital__marital__col_1__marital_A',
 'remainder__marital__marital__col_1__marital_B',
 'remainder__marital__marital__col_1__marital_M',
 'remainder__marital__marital__col_1__marital_S',
 'remainder__marital__marital__col_1__marital_U',
 'remainder__remainder__crclscod__remainder__crclscod',
 'remainder__remainder__asl_flag__remainder__asl_flag',
 'remainder__remainder__refurb_ne

In [ ]:
df_t5.shape

(10000, 110)

In [ ]:
aborrar_=  ['remainder__remainder__creditcd__col_1__creditcd_-U', 'remainder__remainder__creditcd__col_1__creditcd_Y', 'remainder__remainder__remainder__col_num__rev', 'remainder__remainder__remainder__col_num__totmrc', 'remainder__remainder__remainder__col_num__da', 'remainder__remainder__remainder__col_num__ovrmou', 'remainder__remainder__remainder__col_num__ovrrev', 'remainder__remainder__remainder__col_num__vceovr', 'remainder__remainder__remainder__col_num__datovr', 'remainder__remainder__remainder__col_num__roam', 'remainder__remainder__remainder__col_num__change_rev', 'remainder__remainder__remainder__col_num__income', 'remainder__remainder__remainder__remainder__plcd_vce', 'remainder__remainder__remainder__remainder__comp_dat', 'remainder__remainder__remainder__remainder__cc_mou', 'remainder__remainder__remainder__remainder__inonemin', 'remainder__remainder__remainder__remainder__mou_opkd', 'remainder__remainder__remainder__remainder__attempt', 'remainder__remainder__remainder__remainder__comp_vce', 'remainder__remainder__remainder__remainder__totrev', 'remainder__remainder__remainder__remainder__totmou', 'remainder__remainder__remainder__remainder__totcalls']

In [ ]:
df_t5.drop(aborrar_, axis=1, inplace=True)

In [ ]:
df_t5.shape

(10000, 88)

In [ ]:
df_t5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1090001 to 1100000
Data columns (total 88 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   kids__kids__kids__kids                                                    10000 non-null  int64  
 1   remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_UNKW              10000 non-null  int64  
 2   remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_WC                10000 non-null  int64  
 3   remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_WCMB              10000 non-null  int64  
 4   remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_C  10000 non-null  int64  
 5   remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_R  10000 non-null  int64  
 6   remainder__prizm_social_one__prizm_social_one__col_1__

In [ ]:
df_t5.shape

(10000, 88)

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [ ]:
#orden de las variables
columnas = df_t5.columns
for i in columnas:
  print(i)

kids__kids__kids__kids
remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_UNKW
remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_WC
remainder__hnd_web_cap__hnd_webcap__col_mas__hnd_webcap_WCMB
remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_C
remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_R
remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_S
remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_T
remainder__prizm_social_one__prizm_social_one__col_1__prizm_social_one_U
remainder__marital__marital__col_1__marital_A
remainder__marital__marital__col_1__marital_B
remainder__marital__marital__col_1__marital_M
remainder__marital__marital__col_1__marital_S
remainder__marital__marital__col_1__marital_U
remainder__remainder__crclscod__remainder__crclscod
remainder__remainder__asl_flag__remainder__asl_flag
remainder__remainder__refurb_new__remainder__refurb_new
remainder__remainder__area__col_mas__area

# Rescaling

* Si se entrenó el modelo con un dataset estandarizado, estandarizar con mismo scaler.

# PREDICT

* predict() y predict_proba()

In [ ]:
# Check features matched features model
features_predict = list(df_t5.columns)
model_features = list(model.feature_names_in_)

In [ ]:
features_predict == model_features

False

In [ ]:
predict = model.predict(df_t5)

In [ ]:
predict

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
#probabilidad
predict_proba = model.predict_proba(df_t5)[:,1]

In [ ]:
predict_proba

array([0.22487044, 0.39718186, 0.25928608, ..., 0.91857156, 0.67913227,
       0.71620392])

# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [ ]:
pd.DataFrame(predict).to_csv(PATH_DATA+'churn_prediction.csv', index=True)